# Semi Supervised (Sklearn) Demo

Developed by Jhonnatan Torres on Jan 12th, 2022

Version 1
___

**Goal**: Compare the performance of different *semi supervised* estimators available in [**scikit-learn**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.semi_supervised)

## Data Preparation

Importing required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import LabelPropagation, LabelSpreading, SelfTrainingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import classification_report, accuracy_score

Using the *breast cancer* dataset for this demo

In [2]:
X, y = load_breast_cancer(as_frame=True, return_X_y=True)

In [3]:
X.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [4]:
y.value_counts()

1    357
0    212
Name: target, dtype: int64

In [5]:
y.value_counts(normalize=True)

1    0.627417
0    0.372583
Name: target, dtype: float64

In [6]:
ss_index = X.sample(frac=0.5, random_state=1234).index

In [7]:
y_ss = y.copy()

In order to use the *semi supervised* estimators, the unknown labels should have the value **-1** and for this demo, it is assigned to **50%** of the overall records

In [8]:
y_ss.loc[ss_index] = -1

In [9]:
## The goal is to predict the labels with a "-1" value
y_ss.values

array([-1,  0,  0,  0,  0,  0, -1, -1,  0,  0,  0, -1,  0, -1,  0,  0,  0,
       -1,  0,  1, -1, -1, -1,  0, -1, -1,  0, -1,  0, -1,  0,  0,  0, -1,
       -1,  0, -1, -1,  0,  0, -1,  0,  0, -1, -1,  0,  1, -1,  1,  1, -1,
       -1,  1,  0, -1, -1,  0, -1, -1, -1,  1,  1,  0,  1, -1,  0, -1, -1,
        1, -1, -1, -1,  0, -1, -1,  0,  1, -1, -1,  1,  1,  1,  0,  0,  1,
       -1,  0,  0,  1, -1, -1, -1, -1, -1,  0, -1,  1, -1,  1, -1, -1, -1,
       -1,  1, -1,  0,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  0, -1,
        0,  1,  0, -1, -1, -1, -1,  0,  0, -1, -1,  1, -1,  0, -1, -1,  0,
        1,  1, -1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1, -1,
        1,  1,  1, -1,  1,  1, -1, -1,  0, -1, -1,  0,  1, -1, -1, -1, -1,
        1, -1,  0, -1,  1,  1, -1,  0,  1,  1,  0, -1,  0,  1,  0, -1,  0,
       -1,  1, -1,  0,  1, -1,  0, -1,  1,  0,  0, -1,  0, -1,  0, -1,  0,
        1, -1, -1, -1,  1,  1,  0,  1,  0, -1, -1,  0,  1,  1, -1, -1, -1,
       -1, -1, -1,  1, -1

In [10]:
y_ss.value_counts()

-1    284
 1    176
 0    109
Name: target, dtype: int64

Using the index of the unknown labels to split the data into train and test datasets, the goal is compare the predictions against the actual values

In [11]:
X_train = X.drop(index=ss_index)
X_test = X.loc[ss_index]

y_train = y.drop(index=ss_index)
y_test = y.loc[ss_index]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(285, 30) (284, 30) (285,) (284,)


In [12]:
X_train.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.013080,0.01860,0.013400,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.18600,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.040060,0.03832,0.020580,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.24300,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.074580,0.05661,0.018670,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.25750,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.024610,0.05688,0.018850,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.16250,0.2364,0.07678
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,0.3345,0.8902,2.217,27.19,0.007510,0.033450,0.03672,0.011370,0.02165,0.005082,15.47,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.17410,0.3985,0.12440
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,0.3063,1.0020,2.406,24.32,0.005731,0.035020,0.03553,0.012260,0.02143,0.003749,15.49,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.20600,0.4378,0.10720
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,0.2976,1.5990,2.039,23.94,0.007149,0.072170,0.07743,0.014320,0.01789,0.010080,15.09,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.22100,0.4366,0.20750
10,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,0.05697,0.3795,1.1870,2.466,40.51,0.004029,0.009269,0.01101,0.007591,0.01460,0.003042,19.19,33.88,123.80,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
12,19.17,24.80,132.40,1123.0,0.09740,0.24580,0.20650,0.11180,0.2397,0.07800,0.9555,3.5680,11.070,116.20,0.003139,0.082970,0.08890,0.040900,0.04484,0.012840,20.96,29.94,151.70,1332.0,0.1037,0.3903,0.3639,0.17670,0.3176,0.10230
14,13.73,22.61,93.60,578.3,0.11310,0.22930,0.21280,0.08025,0.2069,0.07682,0.2121,1.1690,2.061,19.21,0.006429,0.059360,0.05501,0.016280,0.01961,0.008093,15.03,32.01,108.80,697.7,0.1651,0.7725,0.6943,0.22080,0.3596,0.14310


In [13]:
X_test.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
531,11.670,20.02,75.21,416.2,0.10160,0.09453,0.04200,0.02157,0.1859,0.06461,0.2067,0.8745,1.393,15.34,0.005251,0.017270,0.01840,0.005298,0.01449,0.002671,13.35,28.81,87.00,550.6,0.1550,0.2964,0.27580,0.08120,0.3206,0.08950
166,10.800,9.71,68.77,357.6,0.09594,0.05736,0.02531,0.01698,0.1381,0.06400,0.1728,0.4064,1.126,11.48,0.007809,0.009816,0.01099,0.005344,0.01254,0.002120,11.60,12.02,73.66,414.0,0.1436,0.1257,0.10470,0.04603,0.2090,0.07699
485,12.450,16.41,82.85,476.7,0.09514,0.15110,0.15440,0.04846,0.2082,0.07325,0.3921,1.2070,5.004,30.19,0.007234,0.074710,0.11140,0.027210,0.03232,0.009627,13.78,21.03,97.82,580.6,0.1175,0.4061,0.48960,0.13420,0.3231,0.10340
66,9.465,21.01,60.11,269.4,0.10440,0.07773,0.02172,0.01504,0.1717,0.06899,0.2351,2.0110,1.660,14.20,0.010520,0.017550,0.01714,0.009333,0.02279,0.004237,10.41,31.56,67.03,330.7,0.1548,0.1664,0.09412,0.06517,0.2878,0.09211
220,13.650,13.16,87.88,568.9,0.09646,0.08711,0.03888,0.02563,0.1360,0.06344,0.2102,0.4336,1.391,17.40,0.004133,0.016950,0.01652,0.006659,0.01371,0.002735,15.34,16.35,99.71,706.2,0.1311,0.2474,0.17590,0.08056,0.2380,0.08718


In [14]:
y_train.value_counts()

1    176
0    109
Name: target, dtype: int64

In [15]:
y_test.value_counts()

1    181
0    103
Name: target, dtype: int64

## Logistic Regression as a baseline

In [16]:
lr_pl = make_pipeline(StandardScaler(), 
                      LogisticRegression(C=0.1, class_weight='balanced', random_state=1234))

In [17]:
lr_pl.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(C=0.1, class_weight='balanced',
                                    random_state=1234))])

In [18]:
lr_pl_preds = lr_pl.predict(X_test)

## Label Propagation

In [19]:
lp_pl = make_pipeline(StandardScaler(), 
                      LabelPropagation(kernel='knn', n_neighbors=13))

Please notice how the entire X was used to fit the estimator, in order to calculate the accuracy, the index of the "-1" labels will be used to compare the actuals against the predictions

In [20]:
lp_pl.fit(X, y_ss)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('labelpropagation',
                 LabelPropagation(kernel='knn', n_neighbors=13))])

In [21]:
lp_pl_preds = lp_pl.predict(X)

## Label Spreading

In [22]:
ls_pl = make_pipeline(StandardScaler(), 
                      LabelSpreading(kernel='knn', n_neighbors=13))

In [23]:
ls_pl.fit(X, y_ss)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('labelspreading',
                 LabelSpreading(kernel='knn', n_neighbors=13))])

In [24]:
ls_pl_preds = ls_pl.predict(X)

## Self Training Classifier

In [25]:
BASE_ESTIMATOR = LogisticRegression(C=0.1, class_weight='balanced', random_state=1234)
stc_pl = make_pipeline(StandardScaler(), 
                      SelfTrainingClassifier(base_estimator=BASE_ESTIMATOR))

In [26]:
stc_pl.fit(X, y_ss)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('selftrainingclassifier',
                 SelfTrainingClassifier(base_estimator=LogisticRegression(C=0.1,
                                                                          class_weight='balanced',
                                                                          random_state=1234)))])

In [27]:
stc_pl_preds = stc_pl.predict(X)

## Final Comparison
Comapring the performance of all the 4 estimators used in this demo

In [28]:
# Baseline Logistic Regression
print(classification_report(y_test, lr_pl_preds))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95       103
           1       0.95      1.00      0.97       181

    accuracy                           0.96       284
   macro avg       0.97      0.95      0.96       284
weighted avg       0.97      0.96      0.96       284



In [29]:
# Baseline Label Propagation
print(classification_report(y_test, lp_pl_preds[ss_index]))

              precision    recall  f1-score   support

           0       1.00      0.83      0.90       103
           1       0.91      1.00      0.95       181

    accuracy                           0.94       284
   macro avg       0.95      0.91      0.93       284
weighted avg       0.94      0.94      0.94       284



In [30]:
# Baseline Label Spreading
print(classification_report(y_test, ls_pl_preds[ss_index]))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91       103
           1       0.91      1.00      0.96       181

    accuracy                           0.94       284
   macro avg       0.96      0.92      0.93       284
weighted avg       0.95      0.94      0.94       284



In [31]:
# Baseline Self Training Classifier
print(classification_report(y_test, stc_pl_preds[ss_index]))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       103
           1       0.95      0.99      0.97       181

    accuracy                           0.96       284
   macro avg       0.97      0.95      0.96       284
weighted avg       0.97      0.96      0.96       284



In [32]:
print("Accuracy Baseline Logistic Regression: {:.3f}".format(accuracy_score(y_test, lr_pl_preds)))
print("Accuracy Label Propagation: {:.3f}".format(accuracy_score(y_test, lp_pl_preds[ss_index])))
print("Accuracy Label Spreading: {:.3f}".format(accuracy_score(y_test, ls_pl_preds[ss_index])))
print("Accuracy Self Training Classifier: {:.3f}".format(accuracy_score(y_test, stc_pl_preds[ss_index])))

Accuracy Baseline Logistic Regression: 0.965
Accuracy Label Propagation: 0.937
Accuracy Label Spreading: 0.940
Accuracy Self Training Classifier: 0.965


In [33]:
print("Mean CV Score Label Propagation: {:.3f}".format(cross_val_score(lp_pl, X, y_ss).mean()))
print("Mean CV Score Label Spreading: {:.3f}".format(cross_val_score(ls_pl, X, y_ss).mean()))
print("Mean CV Score Self Training Classifier: {:.3f}".format(cross_val_score(stc_pl, X, y_ss).mean()))

Mean CV Score Label Propagation: 0.478
Mean CV Score Label Spreading: 0.483
Mean CV Score Self Training Classifier: 0.489


*Low mean CV scores could be related to the "-1" labels used in these estimators*, predictions (1 or 0) are compared against "-1"

## Closure Comments

*   Mostly of the default parameters were used to fit the estimators
*   In terms of performance, both the Baseline (Logistic Regression) and the Self Training Classifier had the same accuracy score
*   Both Label Propagation and Label Spreading estimators had a pretty similar performance
*   At least for this particular dataset, and using 50% of the records as *unknown labels (-1)* the Label Propagation and Spreading estimators had a performance similar to the baseline, in addition, these estimators handled some imbalance (63% - 37% ratio) present in the dataset, without a *class weight* or *sample weight* parameter




